In [0]:
!pip install seaborn --upgrade

# Lecture 12: Spin-spin interactions

As a simple model of a ferromagnetic material, we can consider a set of spins arranged on a lattice. The energy of a pair of spins will be lower if they are aligned in the same direction. To make our analysis simple, we'll assume that only **nearest neighbor** spins are coupled together. Spins that aren't nearest neighbors could also interact, but these interactions will be much weaker due to the greater distance between them.

Mathematically, we can write the energy of a configuration $\underline{\sigma}=\{\sigma_1, \sigma_2, \ldots, \sigma_N\}$ of spins $\sigma_i\in\{-1,1\}$ in a one-dimensional lattice as

$$ 
E(\underline{\sigma}) = -\sum_{i=1}^{N-1} J \sigma_i\,\sigma_{i+1}\,.
$$

The coupling $J$ tells us how strong the interaction is between two neighboring spins.

In our past examples the distribution of a vector of variables $\underline{\sigma}$ was usually a product distribution, but this is no longer the case! Now the state of $\sigma_i$ depends on its neighbors $\sigma_{i\pm 1}$ because it is energetically coupled to them.

### Example: Coding the Gibbs distribution

Like we did in the example with a single spin, let's write a function that returns $P(\underline{\sigma})$. To keep things simple, let's limit ourselves to $N=5$ spins. We'll choose $J=1$ and $T=1$.

In [0]:
import numpy as np
import numpy.random as rng
import seaborn as sns
import matplotlib.pyplot as plt


# Set the coupling and temperature values

J = 1
T = 1


# First, let's just write down the energy function

def E(s1, s2, s3, s4, s5):
    return -J * (s1*s2 + s2*s3 + s3*s4 + s4*s5)

# Second, let's determine the normalizing constant Z
# To do that we have to sum over all possible states!

Z = 0

for s1 in [-1, 1]:
    for s2 in [-1, 1]:
        for s3 in [-1, 1]:
            for s4 in [-1, 1]:
                for s5 in [-1, 1]:
                    Z += np.exp(-E(s1, s2, s3, s4, s5)/T)


# Now that we have the normalization, we can write down the probability

def P(s1, s2, s3, s4, s5):
    """ This function takes the spin configuration as input
        and returns its probability as output """
    
    return np.exp(-E(s1, s2, s3, s4, s5)/T) / Z

If the spins were independent, then the average $\langle \sigma_i \sigma_j \rangle$ would be equal to the product of the magnetizations for each spin, $\langle \sigma_i \rangle \langle \sigma_j \rangle$. However, in this case we know that they're not independent!

The average $\langle \sigma_i \sigma_j \rangle$ is generally referred to as a **correlation**, and $\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$ is called a **connected correlation**. 

Let's explicitly show that the connected correlations are nonzero in an example. First, let's compute the connected correlation for the two neighboring spins $\sigma_1$ and $\sigma_2$. A simple argument by symmetry tells us that $\langle \sigma_1 \rangle = \langle \sigma_2 \rangle = 0$, but what about the pair correlation?

In [2]:
p12 = 0

# Sum over all states -- argh!

for s1 in [-1, 1]:
    for s2 in [-1, 1]:
        for s3 in [-1, 1]:
            for s4 in [-1, 1]:
                for s5 in [-1, 1]:
                    p12 += s1 * s2 * P(s1, s2, s3, s4, s5)

print('The connected correlation is %lf' % p12)

The connected correlation is 0.761594


This makes sense. You might have noticed that the correlation is the same that we got for a single spin in a magnetic field. We could understand this connection by thinking about how $\sigma_1$ is affected by $\sigma_2$: given a specific value for $\sigma_2$, $J \sigma_2$ acts like an *effective magnetic field* to align $\sigma_1$.

What happens if we compute the pair correlation of more distant spins, like $\sigma_1$ and $\sigma_3$?

In [3]:
p13 = 0

# Sum over all states

for s1 in [-1, 1]:
    for s2 in [-1, 1]:
        for s3 in [-1, 1]:
            for s4 in [-1, 1]:
                for s5 in [-1, 1]:
                    p13 += s1 * s3 * P(s1, s2, s3, s4, s5)

print('The connected correlation is %lf' % p13)

The connected correlation is 0.580026


Now the correlation is smaller! In general, the correlation decreases exponentially with the separation between the spins. Let's take a look at the correlation between $\sigma_1$ and $\sigma_5$.

In [4]:
p15 = 0

# Sum over all states

for s1 in [-1, 1]:
    for s2 in [-1, 1]:
        for s3 in [-1, 1]:
            for s4 in [-1, 1]:
                for s5 in [-1, 1]:
                    p15 += s1 * s5 * P(s1, s2, s3, s4, s5)

print('The connected correlation is %lf' % p15)

The connected correlation is 0.336430


This is interesting, but the computations are getting laborious because we need to sum over all configurations of the system. This will only get worse as the system gets larger: the number of configurations is $2^N$, where $N$ is the system size. Is there a better way?

In one dimension we can use some analytical tricks ([transfer matrices](https://en.wikipedia.org/wiki/Transfer-matrix_method), specifically) to solve the model exactly even when the system is infinitely large. But mathematical approaches to understanding the behavior in higher dimensions are tricky.

We might try to randomly sample configurations of the system, but it's not immediately obvious how to do this because 1) the number of configurations becomes enormous when the system is large, and 2) it becomes challenging to compute the partition function $Z$ to normalize the probability. Next time, we'll discuss a way around these challenges.